<a href="https://colab.research.google.com/github/Sahan-Daksh/sdgp-rag-chatbot-test/blob/main/UGC_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install groq
!pip install PyPDF2
!pip install sentence_transformers
!pip install faiss-cpu
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.3 MB/s eta 0:00:00


In [ ]:

#code 2
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai
from groq import Groq

# OpenAI API Key (Replace with your API key)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
# Initialize Groq client
# API Key for Groq
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
client = Groq(api_key=GROQ_API_KEY)

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Step 2: Split text into chunks
def split_text_into_chunks(text, chunk_size=300):
    words = text.split()
    chunks = [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Step 3: Generate embeddings for chunks
def generate_embeddings(text_chunks, model_name="all-MiniLM-L6-v2"):
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text_chunks)
    return embeddings

# Step 4: Store embeddings in FAISS
def create_faiss_index(embeddings):
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 similarity
    index.add(embeddings)
    return index

# Step 5: Retrieve relevant chunks
def retrieve_relevant_chunks(query, text_chunks, index, model):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k=3)  # Retrieve top 3 matches
    return [text_chunks[idx] for idx in indices[0]]

# Step 6: Query the language model
def query_llm(client, query, context):
    messages = [
        {"role": "system", "content": "You are a helpful assistant answering questions based on provided context."},
        {"role": "assistant", "content": context},
        {"role": "user", "content": query},
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
        temperature=0.5,
        max_tokens=512,
        top_p=1,
        stop=None,
        stream=False,
    )
    return response.choices[0].message.content

# Main Code
pdf_path = "/content/drive/MyDrive/UGC.pdf"

# Step 1: Extract text
document_text = extract_text_from_pdf(pdf_path)

# Step 2: Split into chunks
text_chunks = split_text_into_chunks(document_text)

# Step 3: Generate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = generate_embeddings(text_chunks, model_name="all-MiniLM-L6-v2")

# Step 4: Create FAISS index
faiss_index = create_faiss_index(np.array(embeddings))

# Step 5: User Query
user_query = input( "Enteryour ")

# Step 6: Retrieve relevant chunks
relevant_chunks = retrieve_relevant_chunks(user_query, text_chunks, faiss_index, embedding_model)

# Combine chunks into context
context = " ".join(relevant_chunks)

# Step 7: Query the LLM
response = query_llm(client, user_query, context)

print("\nResponse from the model:")
print(response)


Enter your Query......Find details about university admissions for a candidate with the following criteria:  Z-Score: 1.7,   G.C.E. (Advanced Level) Subjects and Grades:   1. Physics - A,   2. Chemistry - A,   3. Biology - B,    Preferred Courses of Study: Medicine, Dental Surgery, Pharmacy,   District: Kandy.    List the eligible courses of study, minimum entry requirements, applicable cut-off marks, and any district quota information for the specified criteria. Include relevant details about preferred courses and universities that offer them.  give all are in a list  witch is in a table.

Response from the model:
Based on the provided criteria, here is the information about university admissions for a candidate from Kandy district with a Z-Score of 1.7 and G.C.E. (Advanced Level) subjects and grades:

| Course of Study | Minimum Entry Requirements | Applicable Cut-off Marks | District Quota Information | Preferred Universities |
| --- | --- | --- | --- | --- |
| Medicine | Z-Score: 1

In [ ]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import spacy
import openai
from groq import Groq
import re
import json

# OpenAI API Key (Replace with your API key)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Initialize Groq client (Replace with your key)
# API Key for Groq
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
client = Groq(api_key=GROQ_API_KEY)

# Initialize NLP for metadata extraction
nlp = spacy.load("en_core_web_sm")

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    print("Extracting text from PDF...")
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Step 2: Split text into structured chunks
def split_text_into_chunks(text, chunk_size=300, overlap=100):
    print("Splitting text into chunks...")
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        chunk = " ".join(words[start:start + chunk_size])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Step 3: Extract metadata (Spacy NER)
def tag_metadata_for_chunks(chunks):
    print("Tagging metadata for chunks...")
    metadata = []
    for idx, chunk in enumerate(chunks):
        doc = nlp(chunk)
        keywords = list(set(ent.text for ent in doc.ents))
        metadata.append({
            "chunk_id": idx,
            "chunk_text": chunk,
            "keywords": keywords
        })
    return metadata

# Step 4: Generate embeddings for text chunks
def generate_embeddings(text_chunks, model_name="all-MiniLM-L6-v2"):
    print("Generating embeddings for chunks...")
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text_chunks, show_progress_bar=True)
    return embeddings, model

# Step 5: Store embeddings in FAISS
def create_faiss_index(embeddings):
    print("Creating FAISS index...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 similarity
    index.add(np.array(embeddings))
    return index

# Step 6: Retrieve relevant chunks using FAISS
def retrieve_relevant_chunks(query, text_chunks, index, model, k=3):
    print("Retrieving relevant chunks...")
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k=k)
    return [text_chunks[idx] for idx in indices[0]]

# Step 7: Query the language model (Groq/LLM)
def query_llm(client, query, context):
    print("Querying the LLM...")
    messages = [
        {"role": "system", "content": "You are a helpful assistant answering questions based on provided context."},
        {"role": "assistant", "content": context},
        {"role": "user", "content": query},
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="llama3-8b-8192",
        temperature=0.3,
        max_tokens=512,
        top_p=1,
        stream=False,
    )
    return response.choices[0].message.content

# Step 8: Save processed data with metadata
def save_preprocessed_data(chunks, metadata, embeddings, output_path="processed_data.json"):
    print("Saving preprocessed data...")
    data = [{"chunk": chunk, "metadata": meta, "embedding": embedding.tolist()}
            for chunk, meta, embedding in zip(chunks, metadata, embeddings)]
    with open(output_path, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4)
    print(f"Data saved to {output_path}")

# Main Function
def main():
    # Input PDF Path
    pdf_path = "/content/drive/MyDrive/UGC.pdf"

    # Step 1: Extract text
    document_text = extract_text_from_pdf(pdf_path)

    # Step 2: Split into chunks
    text_chunks = split_text_into_chunks(document_text, chunk_size=300, overlap=100)

    # Step 3: Tag metadata
    metadata = tag_metadata_for_chunks(text_chunks)

    # Step 4: Generate embeddings
    embeddings, embedding_model = generate_embeddings([meta['chunk_text'] for meta in metadata])

    # Step 5: Create FAISS index
    faiss_index = create_faiss_index(embeddings)

    # Step 6: User Query
    user_query = input("\nEnter your query: ")

    # Step 7: Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(user_query, [meta['chunk_text'] for meta in metadata], faiss_index, embedding_model)
    context = " ".join(relevant_chunks)




    #Details by each chunk- not important for now
    # print("\nRelevant Chunks:")
    # for idx, chunk in enumerate(relevant_chunks):
    #     print(f"\nChunk {idx + 1}:\n{chunk}")

    # Step 8: Query LLM
    llm_response = query_llm(client, user_query, context)
    print("\nResponse from the model:")
    print(llm_response)

    # Step 9: Save processed data
    save_preprocessed_data([meta['chunk_text'] for meta in metadata], metadata, embeddings)

if __name__ == "__main__":
    main()

Extracting text from PDF...
Splitting text into chunks...
Tagging metadata for chunks...
Generating embeddings for chunks...


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Creating FAISS index...

Enter your query: Find details about university admissions for a candidate with the following criteria:  Z-Score: 1.7,   G.C.E. (Advanced Level) Subjects and Grades:   1. Physics - A,   2. Chemistry - A,   3. Biology - B,    Preferred Courses of Study: Medicine, Dental Surgery, Pharmacy,   District: Kandy.    List the eligible courses of study, minimum entry requirements, applicable cut-off marks
Retrieving relevant chunks...
Querying the LLM...

Response from the model:
Based on the provided information, here are the details about university admissions for the candidate:

**Z-Score:** 1.7

**G.C.E. (Advanced Level) Subjects and Grades:**

1. Physics - A
2. Chemistry - A
3. Biology - B

**Preferred Courses of Study:**

1. Medicine
2. Dental Surgery
3. Pharmacy

**District:** Kandy

**Eligible Courses of Study:**

Based on the Z-Score of 1.7, the candidate is eligible to apply for the following courses of study:

1. Medicine
2. Pharmacy

**Minimum Entry Requirem